# Daily Financial News for 6000+ Stocks
### Sentimentanalysis-Modelling of Headlines for Stocks and there prices
@author DHR <br>

## Requirements

In [1]:
#import basic DataScience Modules
#!! for pip-modul-list to install @see requirements.txt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [29]:
#import Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
# import final DataSet for Modelling
data = pd.read_csv('../data/analyst_ratings_processed_final.csv')

## Check relation between Stockprice increase and Sentiment

In [5]:
def checkCorrelation(row):
    if ((row['stockPrice_Change'] != row["senti_Binary"]) and (row['stockPrice_Change'] != 0) and (row["senti_Binary"] !=0)):
        return -1
    if row['stockPrice_Change'] == row["senti_Binary"]:
        return 1
    else: return 0

In [6]:
data["correlation"] = data.apply(checkCorrelation, axis=1)

In [7]:
countRow_Pos = data[data["correlation"] == 1].shape[0]
countRow = data.shape[0]
accuracy = countRow_Pos / countRow
print("Von " + str(countRow) + " Headlines mit Sentiment-Score korrelieren " + str(countRow_Pos) +" positiv mit einem Aktienanstieg.\nDas ergbit eine Accuracy von: " + str(accuracy))

Von 161478 Headlines mit Sentiment-Score korrelieren 61517 positiv mit einem Aktienanstieg.
Das ergbit eine Accuracy von: 0.3809621124859114


In [8]:
countRow_Neg = data[data["correlation"] == -1].shape[0]
countRow = data.shape[0]
accuracy = countRow_Neg / countRow
print("Von " + str(countRow) + " Headlines mit Sentiment-Score korrelieren " + str(countRow_Neg) +" negativ mit einem Aktienanstieg.\nDas ergbit eine Accuracy von: " + str(accuracy))

Von 161478 Headlines mit Sentiment-Score korrelieren 3520 negativ mit einem Aktienanstieg.
Das ergbit eine Accuracy von: 0.02179863510818811


In [10]:
# remove the above columns from the DataFrame
data = data.drop(columns=['correlation'])

## Linear Regression

In [11]:
# Split the DataFrame in Train and Test Data
splitted_set = train_test_split(data, test_size=0.3)
train_set = splitted_set[0]
test_set = splitted_set[1]

In [ ]:
# more var as X 
# train_set = train_set.assign(word_count = lambda dataframe: len(dataframe['headline_cleaned']))
# train_set = train_set.assign(publish_count = lambda dataframe: data[data['stock'] == dataframe['stock']]['id'].count())

In [55]:
X = train_set['senti_score'] # X = train_set[['senti_score', 'publish_count']].values.tolist()
Y = train_set['stockPrice_Change']

X = np.array(list(X)).reshape((-1, 1)) # then doesnt need this
Y = np.array(list(Y))

In [13]:
# Model
model = LinearRegression()
model.fit(X,Y)

LinearRegression()

### Test

In [14]:
# Test Prediction
pred = [[2.375]] # must be 2D
prediction = model.predict(pred)
print('predicted response:', prediction, sep='\n')

predicted response:
[0.01508774]


In [23]:
# Prediction for the DataFrame
test_set["prediction"] = test_set.apply(lambda row: model.predict([[row['senti_score']]])[0], axis=1)

<ipython-input-23-e813cf9a8932>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set["prediction"] = test_set.apply(lambda row: model.predict([[row['senti_score']]])[0], axis=1)


In [22]:
def standardization(pred):
    pred = pred * 100
    if pred < 0.5 and pred > -0.5: return 0
    if pred >= 0.5: return 1
    if pred <= 0.5: return -1
    

In [24]:
# Standardization
test_set["prediction"] = test_set["prediction"].apply(standardization)

<ipython-input-24-6643353d1f58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set["prediction"] = test_set["prediction"].apply(standardization)


### Evaluation

In [25]:
print('intercept:', model.intercept_)
print('slope:', model.coef_)

intercept: -0.0024524959294019554
slope: [0.00738536]


In [26]:
# R2
r_sq = model.score(X, Y)
print('coefficient of determination:', r_sq)

coefficient of determination: 0.00013072397450797357


In [28]:
# Accuracy in test_set
countRow_correct = test_set[test_set['prediction'] == test_set['stockPrice_Change']].shape[0]
countRow = test_set.shape[0]
accuracy = countRow_correct / countRow
print('accuracy:', accuracy)

accuracy: 0.779291553133515


In [36]:
# RMSE in test_set
rmse = mean_squared_error(list(test_set['stockPrice_Change']), list(test_set['prediction']), squared=False)
print('rmse:', rmse)

rmse: 0.4891059923840432
